# 要約 
### Jupyter Notebook 要約

このJupyter Notebookは、「LMSYS - Chatbot Arena」コンペティションにおいて、大規模言語モデル（LLM）であるLlama 3を使用し、ユーザーの好みを予測する問題に取り組んでいます。具体的には、異なるモデルの応答の優劣を評価し、その結果に基づいて予測を行うプロセスを含むノートブックです。

#### 主要な問題:
- ユーザーから提供されたプロンプトに対する2つのチャットボット（モデルAとモデルB）の応答を比較し、どちらが好まれるかを予測すること。

#### 使用した手法:
1. **ライブラリとフレームワーク**:
   - `transformers`: Llamaモデルのトークナイジングやシーケンス分類のために使用。
   - `peft`: 軽量なファインチューニングを可能にするために使用され、LoRa設定を通じてモデルの重みを管理。
   - `torch`: PyTorchを用いて深層学習モデルを構築、GPUでの計算を行う。

2. **データの処理**:
   - Pandasを用いてデータを読み込み、トークナイズすることで、モデルが処理しやすい形式に変換。
   - 特徴量を作成するために、プロンプトや応答の文字列情報を結合して特長抽出を実施。

3. **モデルの構築と推論**:
   - Llamaモデルを2つの異なるGPU（デバイス）に読み込み、それぞれのモデルで予測を行う。
   - データの半分を異なるスレッドで処理することで、推論の効率を向上。

4. **LightGBMとのハイブリッドアプローチ**:
   - LightGBMを使って別途予測を実施し、最終的な予測はLlamaによる予測とLightGBMによる予測の加重平均で得られる。

5. **出力**:
   - 最終的に得られた予測結果をCSVファイル（`submission.csv`）として保存。

このNotebookは、高度な機械学習技術を活用し、ユーザーの好みをより正確に予測するための基盤を提供することを目的としています。特に、異なるモデルの応答を効果的に比較する手法が強調されています。

---


# 用語概説 
以下に、Jupyter Notebookの内容に基づいて、初心者がつまずきそうな専門用語の簡単な解説をリストアップします。

### 専門用語解説

1. **LoRa (Low-Rank Adaptation)**:
   - LoRaは、事前学習済みモデルの適応を効率的に行う技術で、パラメータの更新を低ランク行列に圧縮し、計算コストとメモリ使用量を削減します。これにより、大規模モデルを効率よく調整できるようになります。具体的には、選択的に更新するパラメータの数を減らすことで、少ないリソースでモデルを学習させる手法です。

2. **量子化 (Quantization)**:
   - 量子化は、モデルの数値を低精度な型（例：float32からfloat16）に変換してストレージと処理にかかるコストを削減する技術です。これにより、モデルのサイズが小さくなり、GPUメモリの使用効率が向上します。

3. **アテンションマスク (Attention Mask)**:
   - アテンションマスクは、モデルが入力シーケンスのどの部分に注意を向けるかを制御するための構造です。特にパディングされた部分を無視するために使用され、モデルが無意味な情報を学習しないようにします。

4. **自動混合精度 (Automatic Mixed Precision)**:
   - より少ない計算資源で訓練を行うために、異なる精度のデータを自動で混合して使用する技術です。通常、訓練時に計算が重たくなりがちですが、これにより計算速度が向上し、トレーニングが効率化されます。

5. **トークナイザー (Tokenizer)**:
   - トークナイザーは、テキストデータをモデルが理解できる数値データ（トークン）に変換するためのツールです。この変換は、自然言語処理において不可欠なステップで、文章を単語や文の単位に分割します。

6. **ガーベジコレクション (Garbage Collection)**:
   - プログラミングにおけるメモリ管理の一環で、不要になったオブジェクトや変数を自動的に解放してメモリの効率を改善するプロセスです。Pythonでは、不要なメモリを自動で回収する仕組みがあります。

7. **バッチ処理 (Batch Processing)**:
   - データを小さな単位（バッチ）に分けて処理する方法で、メモリ効率を向上させ、計算速度を早めるために使われる技術です。特に大きなデータセットやモデルでこの方法が重宝されます。

8. **フォールド (Fold)**:
   - クロスバリデーションにおいてデータセットを分割する単位です。フォールドはモデルの訓練と検証を繰り返し行い、モデルの性能を評価するために用いられます。

9. **アンサンブル (Ensemble)**:
   - 複数の異なるモデルの予測を組み合わせて、最終的な予測を決定する手法です。これにより、予測精度を向上させることができます。

これらの説明が、Jupyter Notebookの内容を理解する際に役立つことを願っています。

---


<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## llama3-8b

shout out to:
https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b

</div>
<div class="column-right">

# 日本語訳

## llama3-8b

作成者への感謝:
https://www.kaggle.com/code/kishanvavdara/inference-llama-3-8b


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
```

</div>
<div class="column-right">

# 日本語訳

```python
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/
```

</div>
</details>

In [ ]:
!pip install -q -U bitsandbytes --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U transformers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U tokenizers --no-index --find-links ../input/llm-detect-pip/
!pip install -q -U peft --no-index --find-links ../input/llm-detect-pip/

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python

import torch
import sklearn
import numpy as np
import pandas as pd
import time

from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from torch.cuda.amp import autocast
from threading import Thread

torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)

if (not torch.cuda.is_available()): print("Sorry - GPU required!")

MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'
WEIGHTS_PATH = '/kaggle/input/lmsys-model/model'
MAX_LENGTH = 1024
BATCH_SIZE = 8
DEVICE = torch.device("cuda")    

# # Prepare Data 

test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')

# concatenate strings in list
def process(input_str):
    stripped_str = input_str.strip('[]')
    sentences = [s.strip('"') for s in stripped_str.split('","')]
    return  ' '.join(sentences)

test.loc[:, 'prompt'] = test['prompt'].apply(process)
test.loc[:, 'response_a'] = test['response_a'].apply(process)
test.loc[:, 'response_b'] = test['response_b'].apply(process)

display(sample_sub)
display(test.head(5))

# Prepare text for model
test['text'] = 'User prompt: ' + test['prompt'] +  '\n\nModel A :\n' + test['response_a'] +'\n\n--------\n\nModel B:\n'  + test['response_b']
print(test['text'][0])

# # Tokenize


tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')

tokens = tokenizer(test['text'].tolist(), padding='max_length',
                   max_length=MAX_LENGTH, truncation=True, return_tensors='pt')

INPUT_IDS = tokens['input_ids'].to(DEVICE, dtype=torch.int32)
ATTENTION_MASKS = tokens['attention_mask'].to(DEVICE, dtype=torch.int32)

# Move tensors to CPU and convert them to lists
input_ids_cpu = [tensor.cpu().tolist() for tensor in INPUT_IDS]
attention_masks_cpu = [tensor.cpu().tolist() for tensor in ATTENTION_MASKS]

data = pd.DataFrame()
data['INPUT_IDS'] = input_ids_cpu
data['ATTENTION_MASKS'] = attention_masks_cpu
data[:2]

# # Load model 
# We load 1 model on each gpu.  

# BitsAndBytes configuration
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16,
    bnb_8bit_use_double_quant=False)

# Load base model on GPU 0
device0 = torch.device('cuda:0')

base_model_0 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:0')
base_model_0.config.pad_token_id = tokenizer.pad_token_id

# Load base model on GPU 1
device1 = torch.device('cuda:1')
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:1')
base_model_1.config.pad_token_id = tokenizer.pad_token_id

# Now, we have sucessfully loaded one model on each GPU!

# # Load weights 

# LoRa configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.10,
    bias='none',
    inference_mode=True,
    task_type=TaskType.SEQ_CLS,
    target_modules=['o_proj', 'v_proj'])

# Get peft
model_0 = get_peft_model(base_model_0, peft_config).to(device0) 
# Load weights
model_0.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
model_0.eval()

model_1 = get_peft_model(base_model_1, peft_config).to(device1)
model_1.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)
model_1.eval()

# Trainable Parameters
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()

# # Inference
# 

import gc
gc.collect()

def inference(df, model, device, batch_size=BATCH_SIZE):
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)
    
    generated_class_a = []
    generated_class_b = []
    generated_class_c = []

    model.eval()
    
    for start_idx in range(0, len(df), batch_size):
        end_idx = min(start_idx + batch_size, len(df))
        batch_input_ids = input_ids[start_idx:end_idx].to(device)
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)
        
        with torch.no_grad():
            with autocast():
                outputs = model(
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_mask
                )
        
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()
        
        generated_class_a.extend(probabilities[:, 0])
        generated_class_b.extend(probabilities[:, 1])
        generated_class_c.extend(probabilities[:, 2])
    
    df['winner_model_a'] = generated_class_a
    df['winner_model_b'] = generated_class_b
    df['winner_tie'] = generated_class_c

    torch.cuda.empty_cache()  

    return df

st = time.time()

N_SAMPLES = len(data)

# Split the data into two subsets
half = round(N_SAMPLES / 2)
sub1 = data.iloc[0:half].copy()
sub2 = data.iloc[half:N_SAMPLES].copy()

# Function to run inference in a thread
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)

# Dictionary to store results from threads
results = {}

# start threads
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))

t0.start()
t1.start()

# Wait for all threads to finish
t0.join()
t1.join()

# Combine results back into the original DataFrame
data = pd.concat([results[0], results[1]], axis=0)

print(f"Processing complete. Total time: {time.time() - st}")

# Inference completes in ~4.5 hrs, there are still stuff to improve upon this. I would encourage to try out different post-processing and share. Kaggle way :) 

TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']

sample_sub[TARGETS] = data[TARGETS]
```

</div>
<div class="column-right">

# 日本語訳

```python
import torch
import sklearn
import numpy as np
import pandas as pd
import time

from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from torch.cuda.amp import autocast
from threading import Thread

torch.backends.cuda.enable_mem_efficient_sdp(False)  # メモリ効率の良いSDPを有効にする
torch.backends.cuda.enable_flash_sdp(False)  # フラッシュSDPを無効にする

if (not torch.cuda.is_available()): print("申し訳ありませんが、GPUが必要です！")  # GPUが利用できない場合のメッセージ

MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'  # モデルのパス
WEIGHTS_PATH = '/kaggle/input/lmsys-model/model'  # 重みのパス
MAX_LENGTH = 1024  # トークンの最大長
BATCH_SIZE = 8  # バッチサイズ
DEVICE = torch.device("cuda")  # デバイスをCUDAに設定

# # データの準備

test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータを読み込む
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')  # サンプル提出データを読み込む

# リスト内の文字列を連結する関数
def process(input_str):
    stripped_str = input_str.strip('[]')  # 角括弧を削除
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 文章を分割してクォーテーションを削除
    return  ' '.join(sentences)  # 文章をスペースで連結して返す

test.loc[:, 'prompt'] = test['prompt'].apply(process)  # プロンプトを処理
test.loc[:, 'response_a'] = test['response_a'].apply(process)  # モデルAの応答を処理
test.loc[:, 'response_b'] = test['response_b'].apply(process)  # モデルBの応答を処理

display(sample_sub)  # サンプル提出データを表示
display(test.head(5))  # テストデータの最初の5行を表示

# モデル用のテキストを準備
test['text'] = 'ユーザーのプロンプト: ' + test['prompt'] +  '\n\nモデルA :\n' + test['response_a'] +'\n\n--------\n\nモデルB:\n'  + test['response_b']
print(test['text'][0])  # 最初のテキストを表示

# # トークナイズ

tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')  # トークナイザーの読み込み

tokens = tokenizer(test['text'].tolist(), padding='max_length',
                   max_length=MAX_LENGTH, truncation=True, return_tensors='pt')  # テキストをトークナイズ

INPUT_IDS = tokens['input_ids'].to(DEVICE, dtype=torch.int32)  # 入力IDをデバイスに転送
ATTENTION_MASKS = tokens['attention_mask'].to(DEVICE, dtype=torch.int32)  # アテンションマスクをデバイスに転送

# テンソルをCPUに移動し、リストに変換
input_ids_cpu = [tensor.cpu().tolist() for tensor in INPUT_IDS]
attention_masks_cpu = [tensor.cpu().tolist() for tensor in ATTENTION_MASKS]

data = pd.DataFrame()  # 新しいデータフレームを作成
data['INPUT_IDS'] = input_ids_cpu  # 入力IDを格納
data['ATTENTION_MASKS'] = attention_masks_cpu  # アテンションマスクを格納
data[:2]  # 最初の2行を表示

# # モデルをロード
# 各GPUに1つのモデルを読み込む

# BitsAndBytesの設定
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,  # 8ビットで読み込む
    bnb_8bit_compute_dtype=torch.float16,  # 8ビット計算のデータ型を指定
    bnb_8bit_use_double_quant=False)  # 二重量子化を無効にする

# GPU 0にベースモデルを読み込む
device0 = torch.device('cuda:0')

base_model_0 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # ラベルの数
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定
    quantization_config=bnb_config,
    device_map='cuda:0')  # モデルをGPU 0にマップ
base_model_0.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンIDを設定

# GPU 1にベースモデルを読み込む
device1 = torch.device('cuda:1')
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:1')  # モデルをGPU 1にマップ
base_model_1.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンIDを設定

# これで、各GPUにモデルが正常に読み込まれました！

# # 重みを読み込む

# LoRa設定
peft_config = LoraConfig(
    r=16,  # LoRaのパラメータ
    lora_alpha=32,  # LoRaのアルファパラメータ
    lora_dropout=0.10,  # LoRaのドロップアウト率
    bias='none',  # バイアスの設定
    inference_mode=True,  # 推論モードを有効にする
    task_type=TaskType.SEQ_CLS,  # タスクの種類
    target_modules=['o_proj', 'v_proj'])  # 対象モジュール

# PEFTを取得
model_0 = get_peft_model(base_model_0, peft_config).to(device0)  # モデルをGPUに転送
# 重みを読み込む
model_0.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)  # 重みの読み込み
model_0.eval()  # 評価モードに設定

model_1 = get_peft_model(base_model_1, peft_config).to(device1)  # モデルをGPUに転送
model_1.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)  # 重みの読み込み
model_1.eval()  # 評価モードに設定

# 学習可能なパラメータの表示
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()

# # 推論
# 

import gc
gc.collect()  # ガーベジコレクションを実行

def inference(df, model, device, batch_size=BATCH_SIZE):
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)  # 入力IDをテンソルに変換
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)  # アテンションマスクをテンソルに変換
    
    generated_class_a = []  # モデルAの結果を格納するリスト
    generated_class_b = []  # モデルBの結果を格納するリスト
    generated_class_c = []  # 同点を格納するリスト

    model.eval()  # 評価モードに設定
    
    for start_idx in range(0, len(df), batch_size):  # バッチごとに繰り返す
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終わりのインデックスを計算
        batch_input_ids = input_ids[start_idx:end_idx].to(device)  # バッチの入力IDをデバイスに転送
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)  # バッチのアテンションマスクをデバイスに転送
        
        with torch.no_grad():  # 勾配計算を無効にする
            with autocast():  # 自動混合精度を使用
                outputs = model(
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_mask
                )
        
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()  # 出力の確率を計算
        
        generated_class_a.extend(probabilities[:, 0])  # モデルAの結果を追加
        generated_class_b.extend(probabilities[:, 1])  # モデルBの結果を追加
        generated_class_c.extend(probabilities[:, 2])  # 同点の結果を追加
    
    df['winner_model_a'] = generated_class_a  # データフレームにモデルAの結果を保存
    df['winner_model_b'] = generated_class_b  # データフレームにモデルBの結果を保存
    df['winner_tie'] = generated_class_c  # データフレームに同点の結果を保存

    torch.cuda.empty_cache()  # CUDAのメモリを空にする

    return df  # 処理したデータフレームを返す

st = time.time()  # 処理開始時間を記録

N_SAMPLES = len(data)  # データのサンプル数

# データを二つのサブセットに分割
half = round(N_SAMPLES / 2)  # サンプルの半分のサイズを計算
sub1 = data.iloc[0:half].copy()  # 最初のサブセット
sub2 = data.iloc[half:N_SAMPLES].copy()  # 2つ目のサブセット

# スレッドで推論を実行する関数
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)  # 結果を保存

# スレッドからの結果を保存するための辞書
results = {}

# スレッドを開始
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))  # モデルAを使用するスレッド
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))  # モデルBを使用するスレッド

t0.start()  # スレッドの開始
t1.start()  # スレッドの開始

# すべてのスレッドが終了するのを待つ
t0.join()
t1.join()

# 結果を元のデータフレームに統合
data = pd.concat([results[0], results[1]], axis=0)

print(f"処理が完了しました。合計時間: {time.time() - st}")  # 処理時間を表示

# 推論が約4.5時間で完了しました。改善の余地があるため、異なる後処理を試して共有することをお勧めします。Kaggleのやり方 :)

TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']  # 予測のターゲット列

sample_sub[TARGETS] = data[TARGETS]  # サンプル提出に結果を追加
```

</div>
</details>

In [ ]:
import torch
import sklearn
import numpy as np
import pandas as pd
import time

from transformers import AutoTokenizer, LlamaModel, LlamaForSequenceClassification, BitsAndBytesConfig
from peft import get_peft_config, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType
from torch.cuda.amp import autocast
from threading import Thread

torch.backends.cuda.enable_mem_efficient_sdp(False)  # メモリ効率の良いSDPを有効にする
torch.backends.cuda.enable_flash_sdp(False)  # フラッシュSDPを無効にする

if (not torch.cuda.is_available()): print("申し訳ありませんが、GPUが必要です！")  # GPUが利用できない場合のメッセージ

MODEL_NAME = '/kaggle/input/llama-3/transformers/8b-chat-hf/1'  # モデルのパス
WEIGHTS_PATH = '/kaggle/input/lmsys-model/model'  # 重みのパス
MAX_LENGTH = 1024  # トークンの最大長
BATCH_SIZE = 8  # バッチサイズ
DEVICE = torch.device("cuda")  # デバイスをCUDAに設定

# # データの準備

test = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/test.csv')  # テストデータを読み込む
sample_sub = pd.read_csv('/kaggle/input/lmsys-chatbot-arena/sample_submission.csv')  # サンプル提出データを読み込む

# リスト内の文字列を連結する関数
def process(input_str):
    stripped_str = input_str.strip('[]')  # 角括弧を削除
    sentences = [s.strip('"') for s in stripped_str.split('","')]  # 文章を分割してクォーテーションを削除
    return  ' '.join(sentences)  # 文章をスペースで連結して返す

test.loc[:, 'prompt'] = test['prompt'].apply(process)  # プロンプトを処理
test.loc[:, 'response_a'] = test['response_a'].apply(process)  # モデルAの応答を処理
test.loc[:, 'response_b'] = test['response_b'].apply(process)  # モデルBの応答を処理

display(sample_sub)  # サンプル提出データを表示
display(test.head(5))  # テストデータの最初の5行を表示

# モデル用のテキストを準備
test['text'] = 'ユーザーのプロンプト: ' + test['prompt'] +  '\n\nモデルA :\n' + test['response_a'] +'\n\n--------\n\nモデルB:\n'  + test['response_b']
print(test['text'][0])  # 最初のテキストを表示

# # トークナイズ

tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/lmsys-model/tokenizer')  # トークナイザーの読み込み

tokens = tokenizer(test['text'].tolist(), padding='max_length',
                   max_length=MAX_LENGTH, truncation=True, return_tensors='pt')  # テキストをトークナイズ

INPUT_IDS = tokens['input_ids'].to(DEVICE, dtype=torch.int32)  # 入力IDをデバイスに転送
ATTENTION_MASKS = tokens['attention_mask'].to(DEVICE, dtype=torch.int32)  # アテンションマスクをデバイスに転送

# テンソルをCPUに移動し、リストに変換
input_ids_cpu = [tensor.cpu().tolist() for tensor in INPUT_IDS]
attention_masks_cpu = [tensor.cpu().tolist() for tensor in ATTENTION_MASKS]

data = pd.DataFrame()  # 新しいデータフレームを作成
data['INPUT_IDS'] = input_ids_cpu  # 入力IDを格納
data['ATTENTION_MASKS'] = attention_masks_cpu  # アテンションマスクを格納
data[:2]  # 最初の2行を表示

# # モデルをロード
# 各GPUに1つのモデルを読み込む

# BitsAndBytesの設定
bnb_config =  BitsAndBytesConfig(
    load_in_8bit=True,  # 8ビットで読み込む
    bnb_8bit_compute_dtype=torch.float16,  # 8ビット計算のデータ型を指定
    bnb_8bit_use_double_quant=False)  # 二重量子化を無効にする

# GPU 0にベースモデルを読み込む
device0 = torch.device('cuda:0')

base_model_0 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,  # ラベルの数
    torch_dtype=torch.float16,  # モデルのデータ型をfloat16に設定
    quantization_config=bnb_config,
    device_map='cuda:0')  # モデルをGPU 0にマップ
base_model_0.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンIDを設定

# GPU 1にベースモデルを読み込む
device1 = torch.device('cuda:1')
base_model_1 = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
    device_map='cuda:1')  # モデルをGPU 1にマップ
base_model_1.config.pad_token_id = tokenizer.pad_token_id  # パディングトークンIDを設定

# これで、各GPUにモデルが正常に読み込まれました！

# # 重みを読み込む

# LoRa設定
peft_config = LoraConfig(
    r=16,  # LoRaのパラメータ
    lora_alpha=32,  # LoRaのアルファパラメータ
    lora_dropout=0.10,  # LoRaのドロップアウト率
    bias='none',  # バイアスの設定
    inference_mode=True,  # 推論モードを有効にする
    task_type=TaskType.SEQ_CLS,  # タスクの種類
    target_modules=['o_proj', 'v_proj'])  # 対象モジュール

# PEFTを取得
model_0 = get_peft_model(base_model_0, peft_config).to(device0)  # モデルをGPUに転送
# 重みを読み込む
model_0.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)  # 重みの読み込み
model_0.eval()  # 評価モードに設定

model_1 = get_peft_model(base_model_1, peft_config).to(device1)  # モデルをGPUに転送
model_1.load_state_dict(torch.load(WEIGHTS_PATH), strict=False)  # 重みの読み込み
model_1.eval()  # 評価モードに設定

# 学習可能なパラメータの表示
model_0.print_trainable_parameters(), model_1.print_trainable_parameters()

# # 推論
# 

import gc
gc.collect()  # ガーベジコレクションを実行

def inference(df, model, device, batch_size=BATCH_SIZE):
    input_ids = torch.tensor(df['INPUT_IDS'].values.tolist(), dtype=torch.long)  # 入力IDをテンソルに変換
    attention_mask = torch.tensor(df['ATTENTION_MASKS'].values.tolist(), dtype=torch.long)  # アテンションマスクをテンソルに変換
    
    generated_class_a = []  # モデルAの結果を格納するリスト
    generated_class_b = []  # モデルBの結果を格納するリスト
    generated_class_c = []  # 同点を格納するリスト

    model.eval()  # 評価モードに設定
    
    for start_idx in range(0, len(df), batch_size):  # バッチごとに繰り返す
        end_idx = min(start_idx + batch_size, len(df))  # バッチの終わりのインデックスを計算
        batch_input_ids = input_ids[start_idx:end_idx].to(device)  # バッチの入力IDをデバイスに転送
        batch_attention_mask = attention_mask[start_idx:end_idx].to(device)  # バッチのアテンションマスクをデバイスに転送
        
        with torch.no_grad():  # 勾配計算を無効にする
            with autocast():  # 自動混合精度を使用
                outputs = model(
                    input_ids=batch_input_ids,
                    attention_mask=batch_attention_mask
                )
        
        probabilities = torch.softmax(outputs.logits, dim=-1).cpu().numpy()  # 出力の確率を計算
        
        generated_class_a.extend(probabilities[:, 0])  # モデルAの結果を追加
        generated_class_b.extend(probabilities[:, 1])  # モデルBの結果を追加
        generated_class_c.extend(probabilities[:, 2])  # 同点の結果を追加
    
    df['winner_model_a'] = generated_class_a  # データフレームにモデルAの結果を保存
    df['winner_model_b'] = generated_class_b  # データフレームにモデルBの結果を保存
    df['winner_tie'] = generated_class_c  # データフレームに同点の結果を保存

    torch.cuda.empty_cache()  # CUDAのメモリを空にする

    return df  # 処理したデータフレームを返す

st = time.time()  # 処理開始時間を記録

N_SAMPLES = len(data)  # データのサンプル数

# データを二つのサブセットに分割
half = round(N_SAMPLES / 2)  # サンプルの半分のサイズを計算
sub1 = data.iloc[0:half].copy()  # 最初のサブセット
sub2 = data.iloc[half:N_SAMPLES].copy()  # 2つ目のサブセット

# スレッドで推論を実行する関数
def run_inference(df, model, device, results, index):
    results[index] = inference(df, model, device)  # 結果を保存

# スレッドからの結果を保存するための辞書
results = {}

# スレッドを開始
t0 = Thread(target=run_inference, args=(sub1, model_0, device0, results, 0))  # モデルAを使用するスレッド
t1 = Thread(target=run_inference, args=(sub2, model_1, device1, results, 1))  # モデルBを使用するスレッド

t0.start()  # スレッドの開始
t1.start()  # スレッドの開始

# すべてのスレッドが終了するのを待つ
t0.join()
t1.join()

# 結果を元のデータフレームに統合
data = pd.concat([results[0], results[1]], axis=0)

print(f"処理が完了しました。合計時間: {time.time() - st}")  # 処理時間を表示

# 推論が約4.5時間で完了しました。改善の余地があるため、異なる後処理を試して共有することをお勧めします。Kaggleのやり方 :)

TARGETS = ['winner_model_a', 'winner_model_b', 'winner_tie']  # 予測のターゲット列

sample_sub[TARGETS] = data[TARGETS]  # サンプル提出に結果を追加

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
llama_preds = data[TARGETS].values
```

</div>
<div class="column-right">

# 日本語訳

```python
llama_preds = data[TARGETS].values  # ターゲットの値を取得
```

</div>
</details>

In [ ]:
llama_preds = data[TARGETS].values  # ターゲットの値を取得

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## LGBM + tfidf

</div>
<div class="column-right">

# 日本語訳

## LGBM + tfidf


</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
TAG = 'lmsys-chatbot-arena'

import os
RUNPOD = os.path.exists('/workspace/')
KAGGLE = not RUNPOD
if KAGGLE: print('kaggle')
```

</div>
<div class="column-right">

# 日本語訳

```python
TAG = 'lmsys-chatbot-arena'  # コンペティションのタグ

import os
RUNPOD = os.path.exists('/workspace/')  # 実行環境がWORKSPACEかどうか確認
KAGGLE = not RUNPOD  # Kaggleでの実行かどうか確認
if KAGGLE: print('kaggle')  # Kaggleでの実行メッセージ
```

</div>
</details>

In [ ]:
TAG = 'lmsys-chatbot-arena'  # コンペティションのタグ

import os
RUNPOD = os.path.exists('/workspace/')  # 実行環境がWORKSPACEかどうか確認
KAGGLE = not RUNPOD  # Kaggleでの実行かどうか確認
if KAGGLE: print('kaggle')  # Kaggleでの実行メッセージ

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
try:
    import pandas as pd
except:
    !pip install -q kaggle
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm 
    !pip install -q protobuf 
    !pip install -q numba
    
```

</div>
<div class="column-right">

# 日本語訳

```python
try:
    import pandas as pd  # pandasをインポート
except:
    !pip install -q kaggle  # kaggleがインストールされていない場合インストール
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm  # 必要なパッケージをインストール
    !pip install -q protobuf  # protobufをインストール
    !pip install -q numba  # numbaをインストール
```

</div>
</details>

In [ ]:
try:
    import pandas as pd  # pandasをインポート
except:
    !pip install -q kaggle  # kaggleがインストールされていない場合インストール
    !pip install -q pandas matplotlib scipy joblib scikit-learn lightgbm  # 必要なパッケージをインストール
    !pip install -q protobuf  # protobufをインストール
    !pip install -q numba  # numbaをインストール

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)

import os

if RUNPOD:
    if not os.path.exists('~/.kaggle/kaggle.json'):
        !mkdir -p ~/.kaggle
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json
        !chmod 600 /root/.kaggle/kaggle.json

    if not os.path.exists('/workspace/' + TAG + '.zip'):
        !kaggle competitions download $TAG -p /workspace/ 
        
    if not os.path.exists('/data/'):
        import zipfile
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')    
```

</div>
<div class="column-right">

# 日本語訳

```python
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)  # データのパスを設定

import os

if RUNPOD:
    if not os.path.exists('~/.kaggle/kaggle.json'):  # kaggle.jsonが存在しない場合
        !mkdir -p ~/.kaggle  # .kaggleディレクトリを作成
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json  # kaggle.jsonをコピー
        !chmod 600 /root/.kaggle/kaggle.json  # permissionsを変更

    if not os.path.exists('/workspace/' + TAG + '.zip'):  # ZIPファイルが存在しない場合
        !kaggle competitions download $TAG -p /workspace/  # コンペティションデータをダウンロード
        
    if not os.path.exists('/data/'):  # /data/ディレクトリが存在しない場合
        import zipfile
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')  # ZIPファイルを解凍
```

</div>
</details>

In [ ]:
DATA = '/data/' if RUNPOD else 'data/' \
        if not os.path.exists('/kaggle/') \
            else '/kaggle/input/{}/'.format(TAG)  # データのパスを設定

import os

if RUNPOD:
    if not os.path.exists('~/.kaggle/kaggle.json'):  # kaggle.jsonが存在しない場合
        !mkdir -p ~/.kaggle  # .kaggleディレクトリを作成
        !cp /workspace/kaggle.json ~/.kaggle/kaggle.json  # kaggle.jsonをコピー
        !chmod 600 /root/.kaggle/kaggle.json  # permissionsを変更

    if not os.path.exists('/workspace/' + TAG + '.zip'):  # ZIPファイルが存在しない場合
        !kaggle competitions download $TAG -p /workspace/  # コンペティションデータをダウンロード
        
    if not os.path.exists('/data/'):  # /data/ディレクトリが存在しない場合
        import zipfile
        zipfile.ZipFile('/workspace/' + TAG + '.zip').extractall('/data/')  # ZIPファイルを解凍

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
INPUT_PATH = '/kaggle/input/'  
MODEL_PATH = '/workspace/models/'; LOGITS_PATH = '/workspace/logits/'
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'
# MODEL_PATH = MODEL_PATH if not KAGGLE else ''#MODEL_PATH + os.listdir(MODEL_PATH)[0] + '/'
print(MODEL_PATH)

CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'
SAVE_PATH = MODEL_PATH if not KAGGLE else ''
```

</div>
<div class="column-right">

# 日本語訳

```python
INPUT_PATH = '/kaggle/input/'  # 入力パス
MODEL_PATH = '/workspace/models/'; LOGITS_PATH = '/workspace/logits/'  # モデルとロジットのパス
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'  # モデルパスを設定
# MODEL_PATH = MODEL_PATH if not KAGGLE else ''#MODEL_PATH + os.listdir(MODEL_PATH)[0] + '/'
print(MODEL_PATH)  # モデルパスを表示

CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'  # コードパスを設定
SAVE_PATH = MODEL_PATH if not KAGGLE else ''  # 保存パスを設定
```

</div>
</details>

In [ ]:
INPUT_PATH = '/kaggle/input/'  # 入力パス
MODEL_PATH = '/workspace/models/'; LOGITS_PATH = '/workspace/logits/'  # モデルとロジットのパス
MODEL_PATH = MODEL_PATH if not KAGGLE else '/kaggle/input/' \
                + [e for e in os.listdir('/kaggle/input') if 'lsys-models' in e][0] + '/'  # モデルパスを設定
# MODEL_PATH = MODEL_PATH if not KAGGLE else ''#MODEL_PATH + os.listdir(MODEL_PATH)[0] + '/'
print(MODEL_PATH)  # モデルパスを表示

CODE_PATH = MODEL_PATH if KAGGLE else '/workspace/'  # コードパスを設定
SAVE_PATH = MODEL_PATH if not KAGGLE else ''  # 保存パスを設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import os
import io
import gc
import time
import json
import random
import pickle
import zipfile
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from collections import Counter
from collections import defaultdict
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import log_loss
import tokenizers

os.environ['TOKENIZERS_PARALLELISM'] = 'false'
```

</div>
<div class="column-right">

# 日本語訳

```python
import os
import io
import gc
import time
import json
import random
import pickle
import zipfile
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from collections import Counter
from collections import defaultdict
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import log_loss
import tokenizers

os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # TOKENIZERSの並列処理を無効にする
```

</div>
</details>

In [ ]:
import os
import io
import gc
import time
import json
import random
import pickle
import zipfile
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
from collections import Counter
from collections import defaultdict
import torch
from torch import nn
import torch.nn.functional as F
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import log_loss
import tokenizers

os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # TOKENIZERSの並列処理を無効にする

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
train = pd.read_csv(open(DATA + 'train.csv', 'r'))
test = pd.read_csv(open(DATA + 'test.csv', 'r'))
sample = pd.read_csv(DATA + 'sample_submission.csv')

print(len(train), len(test))
```

</div>
<div class="column-right">

# 日本語訳

```python
train = pd.read_csv(open(DATA + 'train.csv', 'r'))  # 訓練データを読み込む
test = pd.read_csv(open(DATA + 'test.csv', 'r'))  # テストデータを読み込む
sample = pd.read_csv(DATA + 'sample_submission.csv')  # サンプル提出データを読み込む

print(len(train), len(test))  # 訓練データとテストデータの長さを表示
```

</div>
</details>

In [ ]:
train = pd.read_csv(open(DATA + 'train.csv', 'r'))  # 訓練データを読み込む
test = pd.read_csv(open(DATA + 'test.csv', 'r'))  # テストデータを読み込む
sample = pd.read_csv(DATA + 'sample_submission.csv')  # サンプル提出データを読み込む

print(len(train), len(test))  # 訓練データとテストデータの長さを表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
params = {}
if False:#len(test) < 10: 
    pass;
    params['subsample'] = 30
else:
    # params['subsample'] = 2
    params['fold'] = -1


params['n_epochs'] = 1
params['n_lgb'] = 1
params['model'] = 'microsoft/deberta-v3-small'
```

</div>
<div class="column-right">

# 日本語訳

```python
params = {}  # パラメータの初期化
if False:  # もしテストデータの長さが10未満の場合
    pass;
    params['subsample'] = 30  # サブサンプルの数を設定
else:
    # params['subsample'] = 2
    params['fold'] = -1  # フォールドを設定


params['n_epochs'] = 1  # 学習のエポック数
params['n_lgb'] = 1  # LightGBMの数
params['model'] = 'microsoft/deberta-v3-small'  # 使用するモデル
```

</div>
</details>

In [ ]:
params = {}  # パラメータの初期化
if False:  # もしテストデータの長さが10未満の場合
    pass;
    params['subsample'] = 30  # サブサンプルの数を設定
else:
    # params['subsample'] = 2
    params['fold'] = -1  # フォールドを設定


params['n_epochs'] = 1  # 学習のエポック数
params['n_lgb'] = 1  # LightGBMの数
params['model'] = 'microsoft/deberta-v3-small'  # 使用するモデル

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# params = {}
FULL = params.get('fold', 0) < 0
N_FOLDS = int(params.get('n_folds', 3)); 
FOLD = int(params.get('fold', 0))
SEED = int(params.get('seed', 3))
SS = int(params.get('subsample', 1))

print(N_FOLDS, FOLD, SEED, SS)

```

</div>
<div class="column-right">

# 日本語訳

```python
# params = {}
FULL = params.get('fold', 0) < 0  # 完全に学習するかどうか
N_FOLDS = int(params.get('n_folds', 3));  # フォールド数を設定
FOLD = int(params.get('fold', 0))  # 現在のフォールドを設定
SEED = int(params.get('seed', 3))  # 乱数シードを設定
SS = int(params.get('subsample', 1))  # サブサンプルの数を設定

print(N_FOLDS, FOLD, SEED, SS)  # フォールド数、現在のフォールド、シード、サブサンプル数を表示
```

</div>
</details>

In [ ]:
# params = {}
FULL = params.get('fold', 0) < 0  # 完全に学習するかどうか
N_FOLDS = int(params.get('n_folds', 3));  # フォールド数を設定
FOLD = int(params.get('fold', 0))  # 現在のフォールドを設定
SEED = int(params.get('seed', 3))  # 乱数シードを設定
SS = int(params.get('subsample', 1))  # サブサンプルの数を設定

print(N_FOLDS, FOLD, SEED, SS)  # フォールド数、現在のフォールド、シード、サブサンプル数を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
from sklearn.model_selection import StratifiedKFold

def get_folds(train): 
    return list(StratifiedKFold(N_FOLDS, random_state = SEED, shuffle = True)\
                    .split(X = np.zeros(len(train)), y = train.iloc[:, -3:].idxmax(1)))

train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]
if SS > 1: train_ids, test_ids = train_ids[::SS], test_ids[::SS]

print(len(train_ids), len(test_ids));  assert set(train_ids) & set(test_ids) == set() 
```

</div>
<div class="column-right">

# 日本語訳

```python
from sklearn.model_selection import StratifiedKFold  # StratifiedKFoldをインポート

def get_folds(train): 
    return list(StratifiedKFold(N_FOLDS, random_state = SEED, shuffle = True)\
                    .split(X = np.zeros(len(train)), y = train.iloc[:, -3:].idxmax(1)))  # フォールドを取得

train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]  # IDを取得
if SS > 1: train_ids, test_ids = train_ids[::SS], test_ids[::SS]  # サブサンプルを取得

print(len(train_ids), len(test_ids));  assert set(train_ids) & set(test_ids) == set()  # 確認
```

</div>
</details>

In [ ]:
from sklearn.model_selection import StratifiedKFold  # StratifiedKFoldをインポート

def get_folds(train): 
    return list(StratifiedKFold(N_FOLDS, random_state = SEED, shuffle = True)\
                    .split(X = np.zeros(len(train)), y = train.iloc[:, -3:].idxmax(1)))  # フォールドを取得

train_ids, test_ids = get_folds(train)[FOLD] if not FULL else [list(range(len(train))), []]  # IDを取得
if SS > 1: train_ids, test_ids = train_ids[::SS], test_ids[::SS]  # サブサンプルを取得

print(len(train_ids), len(test_ids));  assert set(train_ids) & set(test_ids) == set()  # 確認

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def join_strings(x, ):
    x = ' '.join(['' if e is None else e for e in x]) if isinstance(x, list) else x
    return x
```

</div>
<div class="column-right">

# 日本語訳

```python
def join_strings(x, ):
    x = ' '.join(['' if e is None else e for e in x]) if isinstance(x, list) else x  # リスト内の要素を結合
    return x  # 結合した文字列を返す
```

</div>
</details>

In [ ]:
def join_strings(x, ):
    x = ' '.join(['' if e is None else e for e in x]) if isinstance(x, list) else x  # リスト内の要素を結合
    return x  # 結合した文字列を返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def len_join_strings(x, ):
    return len(join_strings(x).split())
```

</div>
<div class="column-right">

# 日本語訳

```python
def len_join_strings(x, ):
    return len(join_strings(x).split())  # 結合した文字列の長さを返す
```

</div>
</details>

In [ ]:
def len_join_strings(x, ):
    return len(join_strings(x).split())  # 結合した文字列の長さを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def len_join_strings_j(x):
    x = json.loads(x)
    return len_join_strings(x)
```

</div>
<div class="column-right">

# 日本語訳

```python
def len_join_strings_j(x):
    x = json.loads(x)  # JSON文字列をロード
    return len_join_strings(x)  # 結合した文字列の長さを返す
```

</div>
</details>

In [ ]:
def len_join_strings_j(x):
    x = json.loads(x)  # JSON文字列をロード
    return len_join_strings(x)  # 結合した文字列の長さを返す

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
torch.manual_seed(datetime.datetime.now().microsecond)
random.seed(datetime.datetime.now().microsecond)
np.random.seed(datetime.datetime.now().microsecond)
```

</div>
<div class="column-right">

# 日本語訳

```python
torch.manual_seed(datetime.datetime.now().microsecond)  # PyTorchの乱数シードを現在のマイクロ秒で設定
random.seed(datetime.datetime.now().microsecond)  # Pythonのrandomモジュールの乱数シードを設定
np.random.seed(datetime.datetime.now().microsecond)  # NumPyの乱数シードを設定
```

</div>
</details>

In [ ]:
torch.manual_seed(datetime.datetime.now().microsecond)  # PyTorchの乱数シードを現在のマイクロ秒で設定
random.seed(datetime.datetime.now().microsecond)  # Pythonのrandomモジュールの乱数シードを設定
np.random.seed(datetime.datetime.now().microsecond)  # NumPyの乱数シードを設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
# TRAIN = True and not KAGGLE
TRAIN = False
INFER = True # or KAGGLE 
SAVE = False
```

</div>
<div class="column-right">

# 日本語訳

```python
# TRAIN = True and not KAGGLE
TRAIN = False  # 訓練フラグを設定
INFER = True  # 推論フラグを設定（KAGGLEの場合も推論）
SAVE = False  # 保存フラグを設定
```

</div>
</details>

In [ ]:
# TRAIN = True and not KAGGLE
TRAIN = False  # 訓練フラグを設定
INFER = True  # 推論フラグを設定（KAGGLEの場合も推論）
SAVE = False  # 保存フラグを設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
import lightgbm as lgb
from sklearn.feature_extraction.text import CountVectorizer
```

</div>
<div class="column-right">

# 日本語訳

```python
import lightgbm as lgb  # LightGBMをインポート
from sklearn.feature_extraction.text import CountVectorizer  # CountVectorizerをインポート
```

</div>
</details>

In [ ]:
import lightgbm as lgb  # LightGBMをインポート
from sklearn.feature_extraction.text import CountVectorizer  # CountVectorizerをインポート

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
LGB = True
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0
INFER_LGB = not TRAIN and LGB
```

</div>
<div class="column-right">

# 日本語訳

```python
LGB = True  # LightGBMフラグを設定
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0  # LightGBMの訓練フラグを設定
INFER_LGB = not TRAIN and LGB  # LightGBMの推論フラグを設定
```

</div>
</details>

In [ ]:
LGB = True  # LightGBMフラグを設定
TRAIN_LGB = TRAIN and LGB and params.get('n_lgb', 1) > 0  # LightGBMの訓練フラグを設定
INFER_LGB = not TRAIN and LGB  # LightGBMの推論フラグを設定

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))
```

</div>
<div class="column-right">

# 日本語訳

```python
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))  # CountVectorizerを読み込み
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))  # カスタムCountVectorizerを読み込み
```

</div>
</details>

In [ ]:
cvec  = pickle.load(open(MODEL_PATH + 'cvec.pkl', 'rb'))  # CountVectorizerを読み込み
ccvec = pickle.load(open(MODEL_PATH + 'ccvec.pkl', 'rb'))  # カスタムCountVectorizerを読み込み

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
def symlog(x): return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)

def dense(x):
    x = np.asarray(x.astype(np.float32).todense())
    x = symlog(x)
    return x

def get_features(df):
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ]
                    for v in [cvec, ccvec]
                ])
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ]
                    for v in [cvec, ccvec]
                ])
    
    v = np.hstack([
    # pfeat, 
          afeat - bfeat, np.abs(afeat - bfeat), 
    # afeat + bfeat
        ])
    try: 
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)
    except: pass

    extras = []
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)
            
    extras.append(df[c].str.len())
    extras.append(df[c].str.split().apply(lambda x: len(x)))
    
    extras = np.stack(extras, axis = 1)
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])
    return np.hstack([v, extras])
    # return v

```

</div>
<div class="column-right">

# 日本語訳

```python
def symlog(x): return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)  # 対称対数変換の関数

def dense(x):
    x = np.asarray(x.astype(np.float32).todense())  # 疎行列を密行列に変換
    x = symlog(x)  # 対称対数変換を適用
    return x  # 変換した値を返す

def get_features(df):
    # プロンプトに対する特徴を抽出
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])
    
    # モデルAに対する特徴を抽出
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ]
                    for v in [cvec, ccvec]
                ])
    
    # モデルBに対する特徴を抽出
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ]
                    for v in [cvec, ccvec]
                ])
    
    # 特徴の組み合わせ
    v = np.hstack([
          afeat - bfeat, np.abs(afeat - bfeat),  # モデルAとBの差分
        ])
    try: 
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)  # モデル数で割る（条件付き）
    except: pass

    extras = []  # 追加の特徴を格納するリスト
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']  # カウントする文字のリスト
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)  # 特徴量としてカウントを追加
            
    extras.append(df[c].str.len())  # 文字列の長さを追加
    extras.append(df[c].str.split().apply(lambda x: len(x)))  # 単語数を追加
    
    extras = np.stack(extras, axis = 1)  # 配列に変換
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])  # 追加特徴の変換
    return np.hstack([v, extras])  # 特徴を結合して返す
    # return v
```

</div>
</details>

In [ ]:
def symlog(x): return (np.sign(x) * np.log1p(np.abs(x))).astype(np.float32)  # 対称対数変換の関数

def dense(x):
    x = np.asarray(x.astype(np.float32).todense())  # 疎行列を密行列に変換
    x = symlog(x)  # 対称対数変換を適用
    return x  # 変換した値を返す

def get_features(df):
    # プロンプトに対する特徴を抽出
    pfeat = np.hstack([dense(v.transform(df[c])) 
                for v in [cvec, ccvec]
                    for c in ['prompt', ]])
    
    # モデルAに対する特徴を抽出
    afeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_a', ]
                    for v in [cvec, ccvec]
                ])
    
    # モデルBに対する特徴を抽出
    bfeat = np.hstack([dense(v.transform(df[c])) 
                for c in ['response_b', ]
                    for v in [cvec, ccvec]
                ])
    
    # 特徴の組み合わせ
    v = np.hstack([
          afeat - bfeat, np.abs(afeat - bfeat),  # モデルAとBの差分
        ])
    try: 
        v = v / (len(all_vote_models) if len(df) < len(train) else 1)  # モデル数で割る（条件付き）
    except: pass

    extras = []  # 追加の特徴を格納するリスト
    EXTRAS = ['\n', '\n\n', '.', ' ', '","']  # カウントする文字のリスト
    for e in EXTRAS:
        for c in ['prompt', 'response_a', 'response_b']:
            extras.append(df[c].str.count(e).values)  # 特徴量としてカウントを追加
            
    extras.append(df[c].str.len())  # 文字列の長さを追加
    extras.append(df[c].str.split().apply(lambda x: len(x)))  # 単語数を追加
    
    extras = np.stack(extras, axis = 1)  # 配列に変換
    extras = np.hstack([extras ** 0.5, np.log1p(extras)])  # 追加特徴の変換
    return np.hstack([v, extras])  # 特徴を結合して返す
    # return v

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))
```

</div>
<div class="column-right">

# 日本語訳

```python
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))  # LightGBMモデルを読み込み
```

</div>
</details>

In [ ]:
lgb_models = pickle.load(open(MODEL_PATH + 'lgb_models.pkl', 'rb'))  # LightGBMモデルを読み込み

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
if INFER and params.get('n_lgb', 1) > 0:
    df = test
    yps = []; b = 1000
    for i in range(0, len(df), b):
        arr = get_features(df.iloc[i: i + b])
        ypms = []
        for model in lgb_models:
            ypms.append(model.predict_proba(arr))
        yps.append(np.stack(ypms).mean(0))
        # break;
        print('.', end = '')
        
        if len(yps) % 2 == 0:
            gc.collect()
    print()

    yp = np.concatenate(yps)
```

</div>
<div class="column-right">

# 日本語訳

```python
if INFER and params.get('n_lgb', 1) > 0:  # 推論が有効で、LightGBMのモデルが存在する場合
    df = test  # テストデータを指定
    yps = []  # 予測を保存するリスト
    b = 1000  # バッチサイズ
    for i in range(0, len(df), b):  # データをバッチごとに処理
        arr = get_features(df.iloc[i: i + b])  # 特徴を取得
        ypms = []  # 各モデルの予測を保存するリスト
        for model in lgb_models:  # すべてのモデルに対して予測を実行
            ypms.append(model.predict_proba(arr))  # モデルからの予測を追加
        yps.append(np.stack(ypms).mean(0))  # モデルの平均を計算してリストに追加
        print('.', end = '')  # 進行状況を表示
        
        if len(yps) % 2 == 0:  # 2つのバッチごとにガーベジコレクションを実行
            gc.collect()
    print()  # 改行

    yp = np.concatenate(yps)  # すべての予測を結合
```

</div>
</details>

In [ ]:
if INFER and params.get('n_lgb', 1) > 0:  # 推論が有効で、LightGBMのモデルが存在する場合
    df = test  # テストデータを指定
    yps = []  # 予測を保存するリスト
    b = 1000  # バッチサイズ
    for i in range(0, len(df), b):  # データをバッチごとに処理
        arr = get_features(df.iloc[i: i + b])  # 特徴を取得
        ypms = []  # 各モデルの予測を保存するリスト
        for model in lgb_models:  # すべてのモデルに対して予測を実行
            ypms.append(model.predict_proba(arr))  # モデルからの予測を追加
        yps.append(np.stack(ypms).mean(0))  # モデルの平均を計算してリストに追加
        print('.', end = '')  # 進行状況を表示
        
        if len(yps) % 2 == 0:  # 2つのバッチごとにガーベジコレクションを実行
            gc.collect()
    print()  # 改行

    yp = np.concatenate(yps)  # すべての予測を結合

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lgb_preds = yp
```

</div>
<div class="column-right">

# 日本語訳

```python
lgb_preds = yp  # LightGBMの予測を保存
```

</div>
</details>

In [ ]:
lgb_preds = yp  # LightGBMの予測を保存

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

## Blend predictions

$\operatorname{preds} = 0.2 \cdot \operatorname{lgbm boosting preds} + 0.8 \cdot \operatorname{llama preds}$


</div>
<div class="column-right">

# 日本語訳

## 予測のブレンド

$\operatorname{preds} = 0.2 \cdot \operatorname{lgbm \ boosting \ preds} + 0.8 \cdot \operatorname{llama \ preds}$



</div>

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
lgb_wt = 0.2 
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds
```

</div>
<div class="column-right">

# 日本語訳

```python
lgb_wt = 0.2  # LightGBMの重みを設定
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds  # 予測をブレンド
```

</div>
</details>

In [ ]:
lgb_wt = 0.2  # LightGBMの重みを設定
preds = lgb_wt * lgb_preds + (1 - lgb_wt) * llama_preds  # 予測をブレンド

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
out = pd.DataFrame(preds, 
                index = df.id, 
                    columns = train.columns[-3:])
display(out.head())
```

</div>
<div class="column-right">

# 日本語訳

```python
out = pd.DataFrame(preds, 
                index = df.id,  # インデックスに元のIDを設定
                    columns = train.columns[-3:])  # 予測列の列名を設定
display(out.head())  # 結果を表示
```

</div>
</details>

In [ ]:
out = pd.DataFrame(preds, 
                index = df.id,  # インデックスに元のIDを設定
                    columns = train.columns[-3:])  # 予測列の列名を設定
display(out.head())  # 結果を表示

<details>
  <summary>pythonコードの比較（クリックすると展開されます）</summary>

<style>
.column-left{
  float: left;
  width: 47.5%;
  text-align: left;
}
.column-right{
  float: right;
  width: 47.5%;
  text-align: left;
}
.column-one{
  float: left;
  width: 100%;
  text-align: left;
}
</style>


<div class="column-left">

# original

```python
out.to_csv('submission.csv')
```

</div>
<div class="column-right">

# 日本語訳

```python
out.to_csv('submission.csv')  # 結果をCSVファイルに保存
```

</div>
</details>

In [ ]:
out.to_csv('submission.csv')  # 結果をCSVファイルに保存